## 필요한 패키지 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
!pip install pyter3
!pip install sacrebleu
!pip install -U nltk
!pip install hLepor
!pip install nptyping

     |████████████████████████████████| 45 kB 2.7 MB/s 


In [13]:
!pip install --upgrade pip  # ensures that pip is current
!git clone https://github.com/google-research/bleurt.git
%cd bleurt
!pip install .

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Cloning into 'bleurt'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 116 (delta 32), reused 55 (delta 20), pack-reused 40
Receiving objects: 100% (116/116), 31.27 MiB | 32.15 MiB/s, done.
Resolving deltas: 100% (36/36), done.
/content/bleurt
Processing /content/bleurt
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 352 kB 5.3 MB/s            
     |████████████████████████████████| 1.2 MB 57.1 MB/s            
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456761 sha256=141e9c07e99a712edfcbaa56c6f9b62361631d33d1421a08ead04c1c8199773c
  Stored in directory: /tmp/pip-ephem-wheel-cache-ds29sbiq/wheels/15/d1/f6/2ef9b6f78af64ad60663

In [3]:
import pyter

In [4]:
# tokenized inputs required
ref = 'As a result of multivariate analysis, old age, male, metastatic site, and histological type were important factors for poor prognosis.'.split()
hyp = 'After multivariate analysis, old age, male sex, metastatic site, and histologic type were significant factors for poor prognosis.'.split()
print(pyter.ter(hyp, ref))

0.4


In [14]:
import nltk, sacrebleu
from sacrebleu.metrics import BLEU, CHRF, TER
from nltk import word_tokenize, meteor
from nltk.translate import ribes
import hlepor
from bleurt import score
checkpoint = "bleurt/test_checkpoint"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [8]:
bleu = BLEU()
chrf = CHRF()
ter = TER()

In [9]:
refs = ['As a result of multivariate analysis, old age, male, metastatic site, and histological type were important factors for poor prognosis.']
sys = 'After multivariate analysis, old age, male sex, metastatic site, and histologic type were significant factors for poor prognosis.'

print(bleu.sentence_score(sys, refs))
print(chrf.sentence_score(sys, refs))
print(ter.sentence_score(sys, refs))

It is recommended to enable `effective_order` for sentence-level BLEU.


BLEU = 53.73 82.6/68.2/52.4/40.0 (BP = 0.917 ratio = 0.920 hyp_len = 23 ref_len = 25)
chrF2 = 78.54
TER = 40.00


In [ ]:
# temp = bleu.corpus_score(sys, refs)
# temp.score

53.730043685394435

In [10]:
refs = [word_tokenize('As a result of multivariate analysis, old age, male, metastatic site, and histological type were important factors for poor prognosis.')]
hyp = word_tokenize('After multivariate analysis, old age, male sex, metastatic site, and histologic type were significant factors for poor prognosis.')

In [11]:
meteor(refs, hyp)

0.8050907258064516

In [15]:
references = ["That which we call a rose, by any other name, would smell as sweet."]
candidates = ["What we call a rose would smell as sweet under any other name."]
scorer = score.BleurtScorer(checkpoint)
scores = scorer.score(references=references, candidates=candidates)
# assert type(scores) == list and len(scores) == 1
print(scores) # 0.58 (일반 BLEU로는 0.43)
print(type(scores))

INFO:tensorflow:Reading checkpoint bleurt/test_checkpoint.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint dbleurt_tiny
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:dbleurt_tiny
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.
[0.580745279788971]
<class 'list'>


## 데이터셋 불러오기

In [16]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import json
import time
import ast

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
%matplotlib inline
import seaborn as sns
sns.set(style='white', context='notebook', palette='deep')

warnings.filterwarnings(action='ignore')

In [19]:
# HTER 스코어링 되지 않은 데이터셋 입력
PROJECT_DIR = '/content/drive/MyDrive/Data'
DATA_DIR = PROJECT_DIR + '/ext/data'
os.listdir(DATA_DIR)
RAW_DATA_DIR = DATA_DIR + '/raw'
df = pd.read_excel(RAW_DATA_DIR + '/dataset_1099.xlsx')

In [20]:
df.head()

,no,sn,style,data_set,domain,subdomain,ko,en,en_google,en_google_edit,en_papago,en_papago_edit,en_kakao,en_kakao_edit,en_words,ko_chars
0,1,01a58379-cf97-4627-8b19-ac9c5f8ddda2,문어체,뉴스,정치,노컷뉴스,그게 1달러가 팔천 한 삼백 원 됩니다.,"That's about KRW 8,300 for a dollar.",That's eight thousand and three hundred won fo...,That's about eight thousand and three hundred ...,One dollar is about 800 won.,"For one dollar, it is about 8,300 won.","That's a dollar, eight thousand and three hund...",That's about eight thousand and three hundred ...,7,22
1,2,6cb00b07-1175-492b-b1da-7bf72e63aea0,문어체,뉴스,정치,국민일보,이후 수행단이 함께 참석하는 만찬이 이어질 예정이다.,A banquet with entourages will follow.,"Afterwards, there will be a banquet attended b...","Afterwards, there will be a banquet attended b...","After that, a dinner will continue with the at...","After that, a dinner will continue with the en...",A dinner will be held with the entourage.,"Afterwards, a dinner with the entourage will b...",7,29
2,3,919b43cc-9220-49df-ae5a-f8fbd8aef9c7,문어체,뉴스,경제,한겨레,특히 화장품 판매직 사원은 그 정도가 심하다.,This is especially true for cosmetics salespeo...,"In particular, the number of employees selling...","In particular, it is severe for employees sell...","In particular, cosmetics salespeople are to th...","In particular, it is true for cosmetics salesp...","Especially, cosmetics salespersons are so severe.",It is especially severe for cosmetics salespeo...,7,25
3,4,b118538a-8b0b-4d0e-807b-936a1063b872,문어체,뉴스,경제,전자신문,경영 환경 변화에 초점을 맞출 필요가 있다.,Focusing on changing business conditions is ne...,There is a need to focus on changes in the bus...,There is a need to focus on changes in the bus...,It is necessary to focus on changes in the bus...,It is necessary to focus on changes in the bus...,It is necessary to focus on changes in the man...,It is necessary to focus on changes in the man...,7,24
4,5,SCS51B11452,문어체,사회과학,문화,공연,그의 말은 강렬한 정치적 메시지를 담고 있었다.,His words contained a strong political message.,His words carried a strong political message.,His words contained a strong political message.,His words contained an intense political message.,His words contained an intense political message.,His words contained a strong political message.,His words contained a strong political message.,7,26


In [22]:
# 데이터 정제 과정에서 텍스트의 단어/글자 수가 바뀌었다면, 업데이트
df['en_words'] = df['en'].str.split().str.len()
df['ko_chars'] = df['ko'].str.len()

In [21]:
df['sn'].size

1099

## TER 구하기

In [24]:
# 문장 단위로 TER 업데이트
df['hter_google'] = df.apply(lambda x: 100 - ter.sentence_score(x['en_google'], [x['en_google_edit']]).score, axis=1)
df['hter_papago'] = df.apply(lambda x: 100 - ter.sentence_score(x['en_papago'], [x['en_papago_edit']]).score, axis=1)
df['hter_kakao'] = df.apply(lambda x: 100 - ter.sentence_score(x['en_kakao'], [x['en_kakao_edit']]).score, axis=1)

In [25]:
# 코퍼스 단위로 TER을 확인하고 싶을 경우
temp = list(df['en_google_edit'])
en_google_edit = []
en_google_edit.append(temp)
en_google_mt = list(df['en_google'])

temp = list(df['en_papago_edit'])
en_papago_edit = []
en_papago_edit.append(temp)
en_papago_mt = list(df['en_papago'])

temp = list(df['en_kakao_edit'])
en_kakao_edit = []
en_kakao_edit.append(temp)
en_kakao_mt = list(df['en_kakao'])

print(ter.corpus_score(en_google_mt, en_google_edit))
print(ter.corpus_score(en_papago_mt, en_papago_edit))
print(ter.corpus_score(en_kakao_mt, en_kakao_edit))

TER = 15.24
TER = 4.36
TER = 7.67


## BLEU, chrF, METEOR, hLePOR, RIBES, BLEURT 구하기

In [26]:
df['bleu_google'] = df.apply(lambda x: bleu.sentence_score(x['en_google'], [x['en_google_edit']]).score, axis=1)
df['bleu_papago'] = df.apply(lambda x: bleu.sentence_score(x['en_papago'], [x['en_papago_edit']]).score, axis=1)
df['bleu_kakao'] = df.apply(lambda x: bleu.sentence_score(x['en_kakao'], [x['en_kakao_edit']]).score, axis=1)

It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is 

In [27]:
df['chrf_google'] = df.apply(lambda x: chrf.sentence_score(x['en_google'], [x['en_google_edit']]).score, axis=1)
df['chrf_papago'] = df.apply(lambda x: chrf.sentence_score(x['en_papago'], [x['en_papago_edit']]).score, axis=1)
df['chrf_kakao'] = df.apply(lambda x: chrf.sentence_score(x['en_kakao'], [x['en_kakao_edit']]).score, axis=1)

In [28]:
df['meteor_google'] = df.apply(lambda x: meteor([word_tokenize(x['en_google_edit'])], word_tokenize(x['en_google'])), axis=1)
df['meteor_papago'] = df.apply(lambda x: meteor([word_tokenize(x['en_papago_edit'])], word_tokenize(x['en_papago'])), axis=1)
df['meteor_kakao'] = df.apply(lambda x: meteor([word_tokenize(x['en_kakao_edit'])], word_tokenize(x['en_kakao'])), axis=1)

In [29]:
df['hlepor_google'] = df.apply(lambda x: hlepor.single_hlepor_score(x['en_google_edit'], x['en_google']), axis=1)
df['hlepor_papago'] = df.apply(lambda x: hlepor.single_hlepor_score(x['en_papago_edit'], x['en_papago']), axis=1)
df['hlepor_kakao'] = df.apply(lambda x: hlepor.single_hlepor_score(x['en_kakao_edit'], x['en_kakao']), axis=1)

In [30]:
df['ribes_google'] = df.apply(lambda x: ribes([word_tokenize(x['en_google_edit'])], word_tokenize(x['en_google'])), axis=1)
df['ribes_papago'] = df.apply(lambda x: ribes([word_tokenize(x['en_papago_edit'])], word_tokenize(x['en_papago'])), axis=1)
df['ribes_kakao'] = df.apply(lambda x: ribes([word_tokenize(x['en_kakao_edit'])], word_tokenize(x['en_kakao'])), axis=1)

In [31]:
df['bleurt_google'] = df.apply(lambda x: scorer.score(references=[x['en_google_edit']], candidates=[x['en_google']]), axis=1)
df['bleurt_papago'] = df.apply(lambda x: scorer.score(references=[x['en_papago_edit']], candidates=[x['en_papago']]), axis=1)
df['bleurt_kakao'] = df.apply(lambda x: scorer.score(references=[x['en_kakao_edit']], candidates=[x['en_kakao']]), axis=1)

In [32]:
df['bleurt_google'] = df.apply(lambda x: x['bleurt_google'][0], axis=1)
df['bleurt_papago'] = df.apply(lambda x: x['bleurt_papago'][0], axis=1)
df['bleurt_kakao'] = df.apply(lambda x: x['bleurt_kakao'][0], axis=1)

In [33]:
df.head()

,no,sn,style,data_set,domain,subdomain,ko,en,en_google,en_google_edit,en_papago,en_papago_edit,en_kakao,en_kakao_edit,en_words,ko_chars,hter_google,hter_papago,hter_kakao,bleu_google,bleu_papago,bleu_kakao,chrf_google,chrf_papago,chrf_kakao,meteor_google,meteor_papago,meteor_kakao,hlepor_google,hlepor_papago,hlepor_kakao,ribes_google,ribes_papago,ribes_kakao,bleurt_google,bleurt_papago,bleurt_kakao
0,1,01a58379-cf97-4627-8b19-ac9c5f8ddda2,문어체,뉴스,정치,노컷뉴스,그게 1달러가 팔천 한 삼백 원 됩니다.,"That's about KRW 8,300 for a dollar.",That's eight thousand and three hundred won fo...,That's about eight thousand and three hundred ...,One dollar is about 800 won.,"For one dollar, it is about 8,300 won.","That's a dollar, eight thousand and three hund...",That's about eight thousand and three hundred ...,7,22,90.909091,50.000000,54.545455,83.520521,12.794740,39.235422,87.254709,52.907742,70.961920,0.928079,0.579897,0.750388,0.932420,0.647343,0.805925,0.691186,0.176151,0.252670,0.750903,-0.237810,0.580894
1,2,6cb00b07-1175-492b-b1da-7bf72e63aea0,문어체,뉴스,정치,국민일보,이후 수행단이 함께 참석하는 만찬이 이어질 예정이다.,A banquet with entourages will follow.,"Afterwards, there will be a banquet attended b...","Afterwards, there will be a banquet attended b...","After that, a dinner will continue with the at...","After that, a dinner will continue with the en...",A dinner will be held with the entourage.,"Afterwards, a dinner with the entourage will b...",6,29,100.000000,77.777778,44.444444,100.000000,72.925717,25.682196,100.000000,79.160300,64.937758,0.999711,0.897297,0.796754,1.000000,0.909988,0.822133,1.000000,0.764354,0.203495,0.894200,0.316584,0.472310
2,3,919b43cc-9220-49df-ae5a-f8fbd8aef9c7,문어체,뉴스,경제,한겨레,특히 화장품 판매직 사원은 그 정도가 심하다.,This is especially true for cosmetics salespeo...,"In particular, the number of employees selling...","In particular, it is severe for employees sell...","In particular, cosmetics salespeople are to th...","In particular, it is true for cosmetics salesp...","Especially, cosmetics salespersons are so severe.",It is especially severe for cosmetics salespeo...,7,25,33.333333,41.666667,0.000000,24.808415,15.911260,7.267884,77.248745,62.308400,59.261056,0.775220,0.551471,0.250000,0.837980,0.627434,0.583664,0.180951,0.227165,0.000000,0.671402,0.402026,0.512987
3,4,b118538a-8b0b-4d0e-807b-936a1063b872,문어체,뉴스,경제,전자신문,경영 환경 변화에 초점을 맞출 필요가 있다.,Focusing on changing business conditions is ne...,There is a need to focus on changes in the bus...,There is a need to focus on changes in the bus...,It is necessary to focus on changes in the bus...,It is necessary to focus on changes in the bus...,It is necessary to focus on changes in the man...,It is necessary to focus on changes in the man...,7,24,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,0.999772,0.999711,0.999711,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.891126,0.897777,0.899773
4,5,SCS51B11452,문어체,사회과학,문화,공연,그의 말은 강렬한 정치적 메시지를 담고 있었다.,His words contained a strong political message.,His words carried a strong political message.,His words contained a strong political message.,His words contained an intense political message.,His words contained an intense political message.,His words contained a strong political message.,His words contained a strong political message.,7,26,85.714286,100.000000,100.000000,59.460356,100.000000,100.000000,79.316755,100.000000,100.000000,0.864796,0.999023,0.999023,0.909091,1.000000,1.000000,0.506612,1.000000,1.000000,0.789998,0.923322,0.922527


## 상관계수 구하기 (Pearson's r)

In [43]:
from scipy.stats.stats import pearsonr

In [41]:
# (correlation coefficient, p-value) 출력
print('Pearson: BLEU - HTER')
print('Google', pearsonr(df['bleu_google'], df['hter_google']))
print('Papago', pearsonr(df['bleu_papago'], df['hter_papago']))
print('Kakao', pearsonr(df['bleu_kakao'], df['hter_kakao']))

Pearson: BLEU - HTER
Google (0.9490265794705883, 0.0)
Papago (0.9502884983033465, 0.0)
Kakao (0.9408005560032123, 0.0)


In [36]:
print('Pearson: chrF - HTER')
print('Google', pearsonr(df['chrf_google'], df['hter_google']))
print('Papago', pearsonr(df['chrf_papago'], df['hter_papago']))
print('Kakao', pearsonr(df['chrf_kakao'], df['hter_kakao']))

Pearson: chrF - HTER
Google (0.9194542481278716, 0.0)
Papago (0.9340111006982611, 0.0)
Kakao (0.9161879531172098, 0.0)


In [38]:
print('Pearson: METEOR - HTER')
print('Google', pearsonr(df['meteor_google'], df['hter_google']))
print('Papago', pearsonr(df['meteor_papago'], df['hter_papago']))
print('Kakao', pearsonr(df['meteor_kakao'], df['hter_kakao']))

Pearson: METEOR - HTER
Google (0.9048290034334976, 0.0)
Papago (0.9014752866355518, 0.0)
Kakao (0.9277327412163499, 0.0)


In [39]:
print('Pearson: hLEPOR - HTER')
print('Google', pearsonr(df['hlepor_google'], df['hter_google']))
print('Papago', pearsonr(df['hlepor_papago'], df['hter_papago']))
print('Kakao', pearsonr(df['hlepor_kakao'], df['hter_kakao']))

Pearson: hLEPOR - HTER
Google (0.9136821665528168, 0.0)
Papago (0.8999042485719588, 0.0)
Kakao (0.914626495159837, 0.0)


In [40]:
print('Pearson: RIBES - HTER')
print('Google', pearsonr(df['ribes_google'], df['hter_google']))
print('Papago', pearsonr(df['ribes_papago'], df['hter_papago']))
print('Kakao', pearsonr(df['ribes_kakao'], df['hter_kakao']))

Pearson: RIBES - HTER
Google (0.7973465916499762, 7.942286578518317e-243)
Papago (0.8286262733585097, 1.140651622969794e-278)
Kakao (0.7891105840834997, 1.9948293253967007e-234)


In [42]:
print('Pearson: BLEURT - HTER')
print('Google', pearsonr(df['bleurt_google'], df['hter_google']))
print('Papago', pearsonr(df['bleurt_papago'], df['hter_papago']))
print('Kakao', pearsonr(df['bleurt_kakao'], df['hter_kakao']))

Pearson: BLEURT - HTER
Google (0.739155853866028, 1.6725285814040008e-190)
Papago (0.7418085430328724, 1.4124290960600632e-192)
Kakao (0.7667767677716879, 1.9896719024593657e-213)
